# 05. PyTorch Going Modular Exercises

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
#%%writefile modular_scripts/get_data.py
"""
Contains functionality for creating PyTorch DataLoaders for custom image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """
    Creates training and testing DataLoaders 
    Takes in training and testing directory paths and turns their contents into PyTorch datasets, and then into PyTorch DataLoaders
    
    Parameters:
        train_dir: Path to the training directory
        test_dir: Path to the testing directory
        transform: A Torchvision transform to perform on the training and testing data
        batch_size: Sample size for the batches in the DataLoaders
        num_workers: Number of workers (CPU/GPU cores) per DataLoader
        
    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
    """
    train_data = datasets.ImageFolder(root=train_dir, transform=transform)
    test_data = datasets.ImageFolder(root=test_dir, transform=transform)
    
    train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, num_workers=NUM_WORKERS, shuffle=True)
    test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, num_workers=NUM_WORKERS, shuffle=False)
    
    class_names = train_data.classes
    
    return train_dataloader, test_dataloader, class_names

In [2]:
# Example running of get_data.py
#!python get_data.py

## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [3]:
%%writefile modular_scripts/train_args.py
"""
Trains a PyTorch image classification model.
"""
import os
import argparse
import torch
from pathlib import Path
from torchvision import transforms
from torchmetrics import Accuracy
import data_setup, engine, model_builder, utils

def main():
    parser = argparse.ArgumentParser(description="Training a model with modular scripts with hyperparameters")
    parser.add_argument("--train_dir", default="data/PizzaSteakSushi/train", type=str, help="The directory of the training data")
    parser.add_argument("--test_dir", default="data/PizzaSteakSushi/test", type=str, help="The directory of the testing data")
    parser.add_argument("--num_epochs", default=10, type=int, help="The number of epochs to train for")
    parser.add_argument("--batch_size", default=32, type=int, help="The number of samples per batch")
    parser.add_argument("--hidden_units", default=10, type=int, help="The number of hidden units in the model")
    parser.add_argument("--learning_rate", default=0.001, type=float, help="The learning rate of the model")

    args=parser.parse_args()

    NUM_EPOCHS = args.num_epochs
    BATCH_SIZE = args.batch_size
    HIDDEN_UNITS = args.hidden_units
    LEARNING_RATE = args.learning_rate
    print(f"[INFO] Training for {NUM_EPOCHS} epochs, batch size: {BATCH_SIZE}, hidden units: {HIDDEN_UNITS}, learning rate: {LEARNING_RATE}")

    TRAIN_DIR = args.train_dir
    TEST_DIR = args.test_dir

    data_transform = transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor()
    ])

    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=TRAIN_DIR,
        test_dir=TEST_DIR,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    torch.manual_seed(42)
    model = model_builder.TinyVGG(
        in_shape=3,
        hidden=HIDDEN_UNITS,
        out_shape=len(class_names)
    )

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
    metric_fn = Accuracy(task="multiclass", num_classes=len(class_names))

    engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        metric_fn=metric_fn,
        epochs=NUM_EPOCHS
    )
    
    utils.save_model(
        model=model,
        target_dir="models",
        model_name="05_pytorch_going_modular_tinyvgg_args.pth"
    )
    
if __name__ == "__main__":
    main()

Overwriting modular_scripts/train_args.py


In [4]:
# Example running of train.py
# Instead of importing the files into this Jupyter Notebook, it's ran as a Python command line command
# This simulates running the command in the terminal
# It also circumvents issues with directory paths that would occur when importing the file
!python modular_scripts/train_args.py --num_epochs 5 --batch_size 32 --hidden_units 10 --learning_rate 0.001

[INFO] Training for 5 epochs, batch size: 32, hidden units: 10, learning rate: 0.001
Epoch: 0
--------
Train Loss: 1.1062828302383423, Train Acc: 0.30859375 | Test Loss: 1.0988644361495972, Test Acc: 0.3020833432674408
Epoch: 1
--------
Train Loss: 1.0993874073028564, Train Acc: 0.33203125 | Test Loss: 1.0694524049758911, Test Acc: 0.5416666865348816
Epoch: 2
--------
Train Loss: 1.0866265296936035, Train Acc: 0.36328125 | Test Loss: 1.0801142454147339, Test Acc: 0.4517045319080353
Epoch: 3
--------
Train Loss: 1.085021734237671, Train Acc: 0.37890625 | Test Loss: 1.0572490692138672, Test Acc: 0.59375
Epoch: 4
--------
Train Loss: 1.0626752376556396, Train Acc: 0.3984375 | Test Loss: 1.0673949718475342, Test Acc: 0.4630681574344635
Saving model to: models\05_pytorch_going_modular_tinyvgg_args.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [5]:
%%writefile modular_scripts/predict.py
"""
Predicts the class of a given image using a TinyVGG model
"""
import torch
import torchvision
import argparse
import model_builder
 
parser = argparse.ArgumentParser()
parser.add_argument("--image", help="Target image to predict the class of")
parser.add_argument("--model_path", default="models/05_pytorch_going_modular_tinyvgg_args.pth", 
                        type=str, help="Target model to use for the class prediction")

args = parser.parse_args()

IMG_PATH = args.image
MODEL_PATH = args.model_path
class_names = ["pizza", "steak", "sushi"]

def load_model(path=MODEL_PATH):
    model = model_builder.TinyVGG(
        in_shape=3,
        hidden=10,
        out_shape=3
    )
    model.load_state_dict(torch.load(path))
    return model

def predict_on_image(image_path=IMG_PATH, model_path=MODEL_PATH):
    model = load_model(MODEL_PATH)
    image = torchvision.io.read_image(str(image_path)).type(torch.float32)
    image = image / 255.
    transform = torchvision.transforms.Resize(size=(64, 64))
    image = transform(image)
    
    model.eval()
    with torch.inference_mode():
        pred_logits = model(image.unsqueeze(dim=0))
        pred_probs = torch.softmax(pred_logits, dim=1)
        pred_label = torch.argmax(pred_probs, dim=1)
        pred_label_class = class_names[pred_label]
        
    print(f"[INFO] Pred class: {pred_label_class}, Pred prob: {pred_probs.max():.3f}")
    
if __name__ == "__main__":
    predict_on_image()

Overwriting modular_scripts/predict.py


In [6]:
# Example running of predict.py 
!python modular_scripts/predict.py --image "data/PizzaSteakSushi/test/steak/502076.jpg"

[INFO] Pred class: steak, Pred prob: 0.505
